In [3]:
import csv, os, re, sys
import pandas as pd 
from bs4 import BeautifulSoup, SoupStrainer
TCP = '/Users/amycweng/DH/TCP'
sys.path.append('../')

def findTextTCP(id):
    if re.match('B1|B4',id[0:2]):
        path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    else: 
        if f'{id}.P4.xml' in os.listdir(f'{TCP}/P1{id[0:2]}'):
            path = f'{TCP}/P1{id[0:2]}/{id}.P4.xml'
        elif f'{id}.P4.xml' in os.listdir(f'{TCP}/P2{id[0:2]}'): 
            path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    return path 

In [19]:
metadataFolder = '/Users/amycweng/DH/ECBC-Data-2022/TCP metadata'
entries = []
for csvFile in os.listdir(metadataFolder):
    data = pd.read_csv(os.path.join(metadataFolder,csvFile))
    dates = list(data['date'])
    titles = list(data['title'])
    authors = list(data['author'])
    publishers = list(data['publisher']) 
    pubplaces = list(data['pubplace'])
    subjects = list(data['keywords'])

    for idx,tcpID in enumerate(data['id']):
        if tcpID == "id": continue
        date = re.findall(r'\d+',str(dates[idx]))
        if len(date) == 0: continue
        if int(date[0]) > 1557: continue
        filepath = findTextTCP(tcpID)
        with open(filepath,'r') as file: 
            data = file.read()
        # use soupstrainer to only parse the main body
        tag = SoupStrainer("DIV1")
        soup = BeautifulSoup(data,features="xml",parse_only=tag)
        divs = soup.findAll(attrs={"TYPE": "catechism"})
        if len(divs) > 0:      
            entries.append( {"id": tcpID, 
                            "title": titles[idx],
                            "authors": authors[idx],
                            "publisher": publishers[idx],
                            "pubplace":pubplaces[idx],
                            "subject_headings":subjects[idx],
                            "date":dates[idx]
                            }
            )
    print(csvFile)

P1B0.csv
P1A9.csv
P1A8.csv
P2B2.csv
P2B3.csv
P2B1.csv
P2B0.csv
P2B4.csv
P2A8.csv
P2A9.csv
P2A7.csv
P2A6.csv
P2A4.csv
P2A5.csv
P2A1.csv
P2A0.csv
P2A2.csv
P2A3.csv
P1A6.csv
P1A7.csv
P1A5.csv
P1A4.csv
P1A0.csv
P1A1.csv
P1A3.csv
P1A2.csv


In [20]:
# store relevant metadata in a CSV file 
with open("../assets/preE_catechisms.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=entries[0].keys())
    writer.writeheader()
    writer.writerows(entries)

len(entries)

7

In [22]:
import shutil
for t in entries: 
    t = t["id"]
    path = findTextTCP(t)
    shutil.copy(path,'/Users/amycweng/DH/catechisms')